<a href="https://colab.research.google.com/github/AlexeyProvorov/Generative/blob/master/Attention_Translator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

# === Подготовка данных ===

# Наборы предложений на английском и французском языках
english_sentences = [
    "hello how are you",
    "i am fine thank you",
    "what is your name",
    "my name is chatgpt",
    "nice to meet you"
]

french_sentences = [
    "bonjour comment ça va",
    "je vais bien merci",
    "quel est votre nom",
    "mon nom est chatgpt",
    "ravi de vous rencontrer"
]

# Функция для создания словаря из списка предложений
def build_vocab(sentences):
    """
    Создает словарь для языка, сопоставляя каждое слово с уникальным индексом.
    Начальные токены:
    - <PAD>: заполнение для выравнивания последовательностей
    - <SOS>: начало последовательности
    - <EOS>: конец последовательности
    - <UNK>: неизвестное слово
    """
    vocab = {"<PAD>": 0, "<SOS>": 1, "<EOS>": 2, "<UNK>": 3}
    index = 4  # Начальный индекс для новых слов
    for sentence in sentences:
        for word in sentence.split():
            if word not in vocab:
                vocab[word] = index
                index += 1
    return vocab

# Создаем словари для английского и французского языков
english_vocab = build_vocab(english_sentences)
french_vocab = build_vocab(french_sentences)

# Функция для преобразования предложения в последовательность индексов
def sentence_to_indices(sentence, vocab):
    """
    Преобразует предложение в список индексов на основе словаря.
    Добавляет токен <EOS> в конце последовательности.
    """
    indices = [vocab.get(word, vocab["<UNK>"]) for word in sentence.split()]
    indices.append(vocab["<EOS>"])  # Добавляем токен конца последовательности
    return indices

# Преобразуем все предложения в последовательности индексов
english_sequences = [sentence_to_indices(s, english_vocab) for s in english_sentences]
french_sequences = [sentence_to_indices(s, french_vocab) for s in french_sentences]

# === Параметры модели ===

INPUT_DIM = len(english_vocab)   # Размер словаря входного языка
OUTPUT_DIM = len(french_vocab)   # Размер словаря выходного языка
ENC_EMB_DIM = 256                # Размерность эмбеддингов в кодере
DEC_EMB_DIM = 256                # Размерность эмбеддингов в декодере
HID_DIM = 512                    # Размерность скрытых состояний в RNN

# === Создание датасета и загрузчика данных ===

# Определяем класс датасета для загрузки данных
class TranslationDataset(torch.utils.data.Dataset):
    """
    Класс датасета для машинного перевода.
    """
    def __init__(self, input_sequences, target_sequences):
        self.input_sequences = input_sequences  # Последовательности на входном языке
        self.target_sequences = target_sequences  # Последовательности на целевом языке

    def __len__(self):
        return len(self.input_sequences)

    def __getitem__(self, idx):
        return self.input_sequences[idx], self.target_sequences[idx]

# Создаем экземпляр датасета
dataset = TranslationDataset(english_sequences, french_sequences)

# Создаем загрузчик данных для итерации по датасету
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

# === Реализация модели с механизмом внимания ===

# --- Кодер (Encoder) ---
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim):
        super().__init__()
        # Слой эмбеддингов
        self.embedding = nn.Embedding(input_dim, emb_dim)
        # RNN (GRU) для обработки последовательности
        self.rnn = nn.GRU(emb_dim, hid_dim, batch_first=True)

    def forward(self, src):
        """
        Проводит входную последовательность через кодер.
        src: [batch_size, src_len] - входные последовательности
        """
        # Преобразуем входные индексы в эмбеддинги
        embedded = self.embedding(src)  # [batch_size, src_len, emb_dim]
        # Пропускаем через RNN
        outputs, hidden = self.rnn(embedded)  # outputs: все скрытые состояния, hidden: последнее скрытое состояние
        return outputs, hidden  # outputs: [batch_size, src_len, hid_dim], hidden: [1, batch_size, hid_dim]

# --- Механизм внимания (Attention) ---
class Attention(nn.Module):
    def __init__(self, hid_dim):
        super().__init__()
        # Линейный слой для вычисления энергии
        self.attn = nn.Linear(hid_dim * 2, hid_dim)
        # Вектор контекста
        self.v = nn.Parameter(torch.rand(hid_dim))

    def forward(self, hidden, encoder_outputs):
        """
        Вычисляет веса внимания.
        hidden: [batch_size, hid_dim] - текущее скрытое состояние декодера
        encoder_outputs: [batch_size, src_len, hid_dim] - скрытые состояния кодера
        """
        src_len = encoder_outputs.shape[1]

        # Повторяем скрытое состояние декодера для каждого временного шага входной последовательности
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)  # [batch_size, src_len, hid_dim]

        # Конкатенируем скрытые состояния кодера и декодера и вычисляем энергию
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2)))  # [batch_size, src_len, hid_dim]

        # Транспонируем для умножения с v
        energy = energy.permute(0, 2, 1)  # [batch_size, hid_dim, src_len]

        # Повторяем v для каждого примера в батче
        v = self.v.repeat(encoder_outputs.size(0), 1).unsqueeze(1)  # [batch_size, 1, hid_dim]

        # Вычисляем скалярное произведение между v и энергией
        attention = torch.bmm(v, energy).squeeze(1)  # [batch_size, src_len]

        # Применяем softmax для получения вероятностей
        return torch.softmax(attention, dim=1)  # [batch_size, src_len]

# --- Декодер (Decoder) ---
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, attention):
        super().__init__()
        self.output_dim = output_dim      # Размер словаря выходного языка
        self.attention = attention        # Механизм внимания
        self.embedding = nn.Embedding(output_dim, emb_dim)  # Слой эмбеддингов
        self.rnn = nn.GRU(hid_dim + emb_dim, hid_dim, batch_first=True)  # RNN с учетом контекста внимания
        self.fc_out = nn.Linear(hid_dim * 2 + emb_dim, output_dim)  # Выходной линейный слой

    def forward(self, input, hidden, encoder_outputs):
        """
        Выполняет один шаг декодирования.
        input: [batch_size] - текущий токен на входе декодера
        hidden: [batch_size, hid_dim] - предыдущее скрытое состояние декодера
        encoder_outputs: [batch_size, src_len, hid_dim] - скрытые состояния кодера
        """
        # Добавляем дополнительную размерность для времени
        input = input.unsqueeze(1)  # [batch_size, 1]

        # Преобразуем входные индексы в эмбеддинги
        embedded = self.embedding(input)  # [batch_size, 1, emb_dim]

        # Вычисляем веса внимания
        a = self.attention(hidden, encoder_outputs)  # [batch_size, src_len]

        # Добавляем размерность для матричного умножения
        a = a.unsqueeze(1)  # [batch_size, 1, src_len]

        # Вычисляем контекстный вектор как взвешенную сумму скрытых состояний кодера
        weighted = torch.bmm(a, encoder_outputs)  # [batch_size, 1, hid_dim]

        # Объединяем эмбеддинги и контекстный вектор для подачи в RNN
        rnn_input = torch.cat((embedded, weighted), dim=2)  # [batch_size, 1, emb_dim + hid_dim]

        # Пропускаем через RNN
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))  # output: [batch_size, 1, hid_dim]

        # Удаляем размерность времени
        output = output.squeeze(1)    # [batch_size, hid_dim]
        embedded = embedded.squeeze(1)  # [batch_size, emb_dim]
        weighted = weighted.squeeze(1)  # [batch_size, hid_dim]

        # Прогнозируем следующий токен
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim=1))  # [batch_size, output_dim]

        # Возвращаем прогноз, новое скрытое состояние и веса внимания
        return prediction, hidden.squeeze(0), a.squeeze(1)

# --- Модель Seq2Seq ---
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder  # Кодер
        self.decoder = decoder  # Декодер
        self.device = device    # Устройство (CPU или GPU)

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        """
        Проводит входную последовательность через модель и генерирует выходную последовательность.
        src: [batch_size, src_len] - входные последовательности
        trg: [batch_size, trg_len] - целевые последовательности
        teacher_forcing_ratio: вероятность использования правильного следующего слова в качестве входа
        """
        batch_size = src.shape[0]
        trg_len = trg.shape[1]
        trg_vocab_size = self.decoder.output_dim

        # Подготовим тензор для хранения выходов
        outputs = torch.zeros(batch_size, trg_len, trg_vocab_size).to(self.device)

        # Пропускаем входную последовательность через кодер
        encoder_outputs, hidden = self.encoder(src)

        # Начинаем с токена <SOS> в декодере
        input = trg[:, 0]  # Первый токен в целевой последовательности

        attentions = []  # Список для хранения весов внимания на каждом шаге

        for t in range(1, trg_len):
            # Выполняем один шаг декодирования
            output, hidden, attention = self.decoder(input, hidden, encoder_outputs)

            # Сохраняем прогноз
            outputs[:, t] = output

            # Решаем, использовать ли teacher forcing
            teacher_force = np.random.random() < teacher_forcing_ratio
            top1 = output.argmax(1)  # Индекс слова с наибольшей вероятностью

            # Следующий вход для декодера
            input = trg[:, t] if teacher_force else top1

            # Сохраняем веса внимания
            attentions.append(attention.cpu().detach().numpy())

        # Возвращаем все выходы и веса внимания
        return outputs, attentions

# === Инициализация модели и параметров ===

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Создаем экземпляры кодера, механизма внимания и декодера
attn = Attention(HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, attn)

# Создаем модель Seq2Seq
model = Seq2Seq(enc, dec, device).to(device)

# Определяем функцию потерь и оптимизатор
criterion = nn.CrossEntropyLoss(ignore_index=english_vocab["<PAD>"])  # Игнорируем потери от паддингов
optimizer = optim.Adam(model.parameters())

# === Обучение модели ===

def train(model, dataloader, optimizer, criterion, clip):
    """
    Обучает модель на одном проходе по датасету.
    """
    model.train()  # Переводим модель в режим обучения
    epoch_loss = 0  # Суммарный убыток за эпоху

    for src, trg in dataloader:
        # Преобразуем данные в тензоры и перемещаем на устройство
        src = torch.LongTensor(src).to(device)
        trg = torch.LongTensor(trg).to(device)

        optimizer.zero_grad()  # Обнуляем градиенты

        # Получаем прогнозы от модели
        output, _ = model(src, trg)

        # Преобразуем выходы и целевые последовательности для функции потерь
        output_dim = output.shape[-1]
        output = output[:, 1:].reshape(-1, output_dim)  # Убираем токен <SOS>
        trg = trg[:, 1:].reshape(-1)  # Убираем токен <SOS>

        # Вычисляем убыток
        loss = criterion(output, trg)

        # Обратное распространение ошибки
        loss.backward()

        # Ограничиваем градиенты для предотвращения взрыва градиентов
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        # Обновляем параметры модели
        optimizer.step()

        # Накопление убытка
        epoch_loss += loss.item()

    # Возвращаем средний убыток за эпоху
    return epoch_loss / len(dataloader)

# === Функция для перевода предложения ===

def translate_sentence(model, sentence, english_vocab, french_vocab, max_len=10):
    """
    Переводит входное предложение с использованием обученной модели.
    """
    model.eval()  # Переводим модель в режим оценки

    # Преобразуем предложение в последовательность индексов
    tokens = sentence.split()
    indices = [english_vocab.get(token, english_vocab["<UNK>"]) for token in tokens]
    src_tensor = torch.LongTensor(indices).unsqueeze(0).to(device)  # Добавляем размерность батча

    with torch.no_grad():
        # Получаем скрытые состояния кодера
        encoder_outputs, hidden = model.encoder(src_tensor)

    trg_indices = [french_vocab["<SOS>"]]  # Начинаем с токена <SOS>
    attentions = []  # Список для хранения весов внимания

    for i in range(max_len):
        trg_tensor = torch.LongTensor([trg_indices[-1]]).to(device)  # Текущий вход декодера

        with torch.no_grad():
            # Выполняем один шаг декодирования
            output, hidden, attention = model.decoder(trg_tensor, hidden, encoder_outputs)

        # Получаем индекс слова с наибольшей вероятностью
        pred_token = output.argmax(1).item()

        # Добавляем предсказанный токен в последовательность
        trg_indices.append(pred_token)

        # Сохраняем веса внимания
        attentions.append(attention.cpu().numpy())

        # Останавливаем перевод, если встретили токен <EOS>
        if pred_token == french_vocab["<EOS>"]:
            break

    # Преобразуем индексы в слова
    trg_tokens = [list(french_vocab.keys())[list(french_vocab.values()).index(idx)] for idx in trg_indices[1:]]

    # Возвращаем переведенное предложение и веса внимания
    return ' '.join(trg_tokens), attentions

# === Запуск обучения модели ===

N_EPOCHS = 1000  # Количество эпох
CLIP = 1         # Ограничение градиентов

for epoch in range(N_EPOCHS):
    loss = train(model, dataloader, optimizer, criterion, CLIP)

    # Выводим информацию каждые 100 эпох
    if (epoch + 1) % 100 == 0:
        print(f'Epoch: {epoch + 1}, Loss: {loss:.4f}')

# === Пример перевода ===

sentence = "hello how are you"
translation, attentions = translate_sentence(model, sentence, english_vocab, french_vocab)
print(f"Input Sentence: {sentence}")
print(f"Translated Sentence: {translation}")


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def display_attention(sentence, translation, attentions):
    """
    Визуализирует веса внимания между входным и выходным предложениями.
    """
    # Преобразуем веса внимания в numpy массив
    attention = np.array(attentions)

    # Создаём фигуру для визуализации
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111)

    # Используем heatmap для отображения весов внимания
    sns.heatmap(attention[:len(translation.split()), :len(sentence.split())],
                xticklabels=sentence.split(),
                yticklabels=translation.split(),
                cmap='viridis', ax=ax)

    plt.xlabel('Input Sentence')
    plt.ylabel('Translated Sentence')
    plt.show()

# Вызов функции для визуализации
display_attention(sentence, translation, attentions)
